In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


model = ChatOpenAI(
    model=os.environ.get("MODEL_NAME"),
    temperature=0,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
)

# Build a basic agent

In [2]:
from langchain.agents import create_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='778032dc-29d3-45c6-8437-9b5f4a345b20'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 265, 'total_tokens': 286, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-3c24d71f-56bb-4e80-ab8e-dc0aaa4606b3', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--b6fb5310-340a-4ae5-b0f7-ed928d2c5510-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_2c847dfeb5354285b38aa14b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 265, 'output_tokens': 21, 'total_tokens': 286, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  ToolMessage(content="It's always sunny in sf!", name='get_weath

# Build a real-world agent

In [3]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [4]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime


@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [6]:
from dataclasses import dataclass


# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

In [7]:
from langgraph.checkpoint.memory import InMemorySaver


checkpointer = InMemorySaver()

In [8]:
from langchain.agents.structured_output import ToolStrategy


agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

In [9]:
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])

ResponseFormat(punny_response='Looks like Florida is living up to its rep—sunny side up and not a cloud in sight! Don’t forget the sunscreen, or you’ll be fried like a Floridian alligator at a beach BBQ!', weather_conditions='sunny')


In [10]:
# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])

ResponseFormat(punny_response="You're welcome! Stay cool—or at least as cool as a cucumber in a sauna! 🌞", weather_conditions='sunny')
